In [3]:
import weaviate
from weaviate.auth import Auth

client = weaviate.connect_to_local(auth_credentials=Auth.api_key("admin"))

In [41]:
from weaviate.collections.classes.config import Property, DataType
from weaviate.rbac.models import Permissions

if client.roles.exists("test"):
    client.roles.delete("test")

client.roles.create(
    role_name="test",
    permissions=[
        Permissions.data(
            collection="(Public|Questions)*",
            create=False,
            delete=False,
            read=True,
            update=False
        ),
        Permissions.collections(
            collection="Public*|Questions*",
            read_config=True)
    ]
)

for collection in ["PublicTest", "QuestionsTest", "NoAccessTest"]:
    if client.collections.exists(collection):
        client.collections.delete(collection)

    client.collections.create(
        name=collection,
        properties=[Property(name="test", data_type=DataType.TEXT)]
    )
    client.collections.get(collection).data.insert(
        properties={"test": "Hello World"}
    )



In [42]:
client.users.db.assign_roles(user_id="app", role_names="test")

In [43]:
app_client = weaviate.connect_to_local(auth_credentials=Auth.api_key("app"))


/var/folders/3b/zrvl30f13v7ch2v99gp_4q2c0000gn/T/ipykernel_80620/4002572113.py:1: ResourceWarning: unclosed <socket.socket fd=85, family=30, type=1, proto=6, laddr=('::1', 55414, 0, 0), raddr=('::1', 8080, 0, 0)>
  app_client = weaviate.connect_to_local(auth_credentials=Auth.api_key("app"))


In [44]:
app_client.collections.list_all()

{'PublicTest': _CollectionConfigSimple(name='PublicTest', description=None, generative_config=None, properties=[_Property(name='test', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer=None, vectorizer_configs={})], references=[], reranker_config=None, vectorizer_config=None, vectorizer=None, vector_config={'default': _NamedVectorConfig(vectorizer=_NamedVectorizerConfig(vectorizer=<Vectorizers.NONE: 'none'>, model={}, source_properties=None), vector_index_config=_VectorIndexConfigHNSW(multi_vector=None, quantizer=None, cleanup_interval_seconds=300, distance_metric=<VectorDistances.COSINE: 'cosine'>, dynamic_ef_min=100, dynamic_ef_max=500, dynamic_ef_factor=8, ef=-1, ef_construction=128, filter_strategy=<VectorFilterStrategy.SWEEPING: 'sweeping'>, flat_search_cutoff=40000, max_connections=32, skip=False, vector_c

In [45]:
for o in app_client.collections.get("PublicTest").iterator():
    print(o)
for o in app_client.collections.get("QuestionsTest").iterator():
    print(o)


Object(uuid=_WeaviateUUIDInt('2fa7ee2b-a75b-4f79-968f-54649e340b3c'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'test': 'Hello World'}, references=None, vector={}, collection='PublicTest')
Object(uuid=_WeaviateUUIDInt('c8259445-2fb2-4230-abff-8b31f496cc72'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'test': 'Hello World'}, references=None, vector={}, collection='QuestionsTest')


In [46]:
for o in app_client.collections.get("NoAccessTest").iterator():
    print(o)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/ast.py:52: ResourceWarning: unclosed <socket.socket fd=77, family=30, type=1, proto=6, laddr=('::1', 54168, 0, 0), raddr=('::1', 8080, 0, 0)>
  return compile(source, filename, mode, flags,


InsufficientPermissionsError: forbidden! Unexpected status code: permission denied, with response body: rbac: authorization, forbidden action: user 'app' has insufficient permissions to read_data [[Domain: data, Collection: NoAccessTest, Tenant: *, Object: *]].